
# بخش اول − سوالات تئوری مربوط به تمرین عملی

<div dir=rtl>

پاسخ :

(آ) در شبکه‌های کانولوشنی معمولی، grid sampling به صورت منظم و متناوب اعمال می‌شود. این عمل به صورت یک گرید مرتب با فاصله ثابت بین نقاط اعمال می‌شود، که این نقاط به عنوان نقاط مرکزی برای اعمال عملیات کانولوشن استفاده می‌شوند.
شبکه‌های کانولوشن Deformable با استفاده از انتقال‌دهنده‌های اختصاصی (specialized offsets) بر روی گرید ورودی، می‌توانند نقاطی را برای اعمال فیلترهای کانولوشن به دقت ترتیب دهند. این به مدل اجازه می‌دهد که نقاط مرکزی را بر اساس آموزش‌های خود تغییر دهد و به صورت فعال ، نقاطی را انتخاب کند که بهترین بازدهی را دارند.

(ب)

۱. آموزش Offset:

ابتدا، شبکه با آموزش مکانیزم‌های Deformable ، که به عنوان offset یا جابجایی شناخته می‌شوند، می‌تواند به دقت نقاطی را برای اعمال فیلترهای کانولوشن در نظر بگیرد.
این offset ها یادگیری می شوند و اجازه می‌دهند تا مدل به طور پویا تر و انعطاف پذیرتری به نقاط انتخاب شده برای اعمال فیلترها پاسخ دهد.

۲. تطبیق Receptive Field:

سپس، با استفاده از این offset ها، نقاطی که برای اعمال فیلترها در نظر گرفته می‌شوند، تغییر می‌کنند. به این ترتیب، فیلترها و نقاط مرکزی برای اعمال آنها بر اساس offset های یادگرفته شده تغییر مکان می‌دهند.
این انتقال نقاط به مکان‌های دیگر به مدل اجازه می‌دهد تا بهتر به الگوهای نامنظم و تغییرات هندسی در تصاویر واکنش نشان دهد.

(ج) در واقع در این نوع شبکه ها وقتی که تصویر دارای چرخش زیاد است آن یک تصویر جدید برای شبکه تلقی میشود که گویا مشابه آن را قبلا ندیده است و به همین علت وقتی چرخش تصویر از یک حدی بیشتر می شود با مشکل مواجه می شویم.


(د) به وسیله فرآیند یادگیری محاسبه می شوند.این آفست‌ها برای تغییر مکان نقاط اعمال فیلترهای کانولوشن استفاده می‌شوند، به طوری که فیلترها با استفاده از این آفست‌ها، در مکان‌های مناسب تری اعمال شوند.
در واقع این آفست ها را به عنوان پارامتر قابل یادگیری تعریف میکنیم تا بهینه ترین حالت در طی فرآیند آموزش به دست آید.


# بخش دوم-سوالات عملی

## import libraries and data

In [50]:
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset , Dataset
from torchvision.datasets import CocoDetection
import torch
import torch.nn as nn
from torch.optim import Adam
import torch.optim as optim
import os
from pycocotools.coco import COCO
from PIL import Image
import time
import torchvision.ops
import torch.nn.functional as F

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<div dir=rtl>

ابتدا مطابق با دستورات زیر دیتا ست را دانلود و در گوگل درایو آنزیپ کردیم.

ما فقط دیتای ولیدیشن را استفاده کردیم و آن را به دو بخش ترین و تست تقسیم خواهیم کرد.

```python
!wget http://images.cocodataset.org/zips/val2017.zip
%cd /content
!unzip val2017.zip -d /content/drive/MyDrive/DLhw3/

!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
%cd /content
!unzip annotations_trainval2017.zip -d /content/drive/MyDrive/DLhw3/

```

## define dataset class

<div dir=rtl>
در این کلاس دیتاست دلخواه خودمان را ایجاد میکنیم.

به این صورت که برای هر تصویر یک لیبل به صورت یک بردار وان هات 90 تایی ایجاد میکنیم.چون 90 کلاس داریم و هر کدام که در تصویر موجود بود آن را 1 قرار می دهیم و هر کدام که موجود نبود به جای آن صفر قرار می دهیم.

In [46]:
class CustomCocoDataset(Dataset):
    def __init__(self, root, annFile, transform=None):
        self.root = root
        self.coco = COCO(annFile)
        self.transform = transform
        self.ids = list(sorted(self.coco.imgs.keys()))

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index):
        coco = self.coco
        img_id = self.ids[index]
        ann_ids = coco.getAnnIds(imgIds=img_id)
        anns = coco.loadAnns(ann_ids)

        path = coco.loadImgs(img_id)[0]['file_name']
        img = Image.open(os.path.join(self.root, path)).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)

        labels = [ann['category_id'] for ann in anns]
        labels_tensor = torch.tensor(labels, dtype=torch.long)

        labels_tensor = labels_tensor.to(device)
        return img, labels_tensor

In [55]:
def custom_collate_fn(batch):
    images = []
    batch_labels = []

    for img, labels in batch:
        images.append(img)

        one_hot_labels = torch.zeros(90)
        for label in labels:
            one_hot_labels[label-1] = 1

        batch_labels.append(one_hot_labels)

    images = torch.stack(images)
    batch_labels = torch.stack(batch_labels)

    return images, batch_labels

data_dir = '/content/drive/MyDrive/DLhw3/val2017'
annotation_file = '/content/drive/MyDrive/DLhw3/annotations/instances_val2017.json'

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

coco_dataset = CustomCocoDataset(root=data_dir, annFile=annotation_file, transform=transform)

train_indices = range(1000)
test_indices = range(1000, 1300)

train_dataset = Subset(coco_dataset, train_indices)
test_dataset = Subset(coco_dataset, test_indices)

batch_size=16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)

loading annotations into memory...
Done (t=1.39s)
creating index...
index created!


## model-CNN

In [48]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((14, 14))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.adaptive_pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = torch.sigmoid(x)
        return x


## train CNN

In [56]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cnn_model = CustomCNN(num_classes=90)
cnn_model=cnn_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.0002)

num_epochs = 30

start_time = time.time()
for epoch in range(num_epochs):
    running_loss = 0
    correct=0
    total=0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs=inputs.to(device)
        labels=labels.to(device)

        optimizer.zero_grad()

        outputs = cnn_model(inputs)
        predicted = torch.round(outputs)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    accuracy_train = 100 * correct / (total*90)

    print(f'Epoch [{epoch + 1}], Accuracy_training:{accuracy_train:.4f}%')
    print(f'Epoch [{epoch + 1}], Loss: {running_loss / total:.2}')
end_time = time.time()
execution_time = end_time - start_time
print(f"Total training time: {execution_time:.2f} seconds")

Epoch [1], Accuracy_training:72.3278%
Epoch [1], Loss: 0.79
Epoch [2], Accuracy_training:80.8689%
Epoch [2], Loss: 0.79
Epoch [3], Accuracy_training:83.0378%
Epoch [3], Loss: 0.78
Epoch [4], Accuracy_training:81.5289%
Epoch [4], Loss: 0.78
Epoch [5], Accuracy_training:80.6722%
Epoch [5], Loss: 0.77
Epoch [6], Accuracy_training:80.9544%
Epoch [6], Loss: 0.77
Epoch [7], Accuracy_training:82.8867%
Epoch [7], Loss: 0.77
Epoch [8], Accuracy_training:82.4667%
Epoch [8], Loss: 0.76
Epoch [9], Accuracy_training:83.4178%
Epoch [9], Loss: 0.76
Epoch [10], Accuracy_training:84.1733%
Epoch [10], Loss: 0.75
Epoch [11], Accuracy_training:84.3944%
Epoch [11], Loss: 0.75
Epoch [12], Accuracy_training:86.1122%
Epoch [12], Loss: 0.75
Epoch [13], Accuracy_training:86.0578%
Epoch [13], Loss: 0.75
Epoch [14], Accuracy_training:87.0311%
Epoch [14], Loss: 0.74
Epoch [15], Accuracy_training:87.4956%
Epoch [15], Loss: 0.74
Epoch [16], Accuracy_training:87.5311%
Epoch [16], Loss: 0.74
Epoch [17], Accuracy_train

## test CNN

In [57]:
correct = 0
total = 0
test_loss=0
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        inputs, labels = data
        inputs=inputs.to(device)
        labels=labels.to(device)

        outputs = cnn_model(inputs)
        predicted = torch.round(outputs)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        loss = criterion(outputs, labels)
        test_loss += loss.item()
    accuracy_test = 100 * correct / (total*90)

    print(f'Accuracy_test:{accuracy_test:.4f}%')
    print(f'Loss: {running_loss / total:.2}')


Accuracy_test:91.6148%
Loss: 2.4


## Define DeformableConv2d class

<div dir=rtl>

برای تعریف این کلاس از سورس قرار داده شده در تمرین (https://github.com/developer0hye/PyTorch-Deformable-Convolution-v2) استفاده شد.

در ادامه به توضیح نحوه عملکرد این کلاس  میپردازیم.

به طور خلاصه اگر بخواهیم بگوییم ،ابتدا از یک لایه کانولوشنی معمولی استفاده میکنیم برای تولید آفست ها.به این صورت که هر کدام از پیکسل ها آفست مخصوص خود را دارد.همچنین یک لایه کانولوشنی معمولی نیز به عنوان modulator قرار میدهیم که نشان دهنده شدت اعمال کانولوشن است.

پس از این از ماژول کتابخانه ای torchvision.ops.deform_conv2d استفاده می شود، با استفاده از آفست ها و مدولاتور هایی که به دست آوردیم و همچنین یک لایه کانولوشنی معمولی دیگر که اعمال می شود.

In [58]:
class DeformableConv2d(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size=3,
                 stride=1,
                 padding=1,
                 bias=False):

        super(DeformableConv2d, self).__init__()

        self.padding = padding

        self.offset_conv = nn.Conv2d(in_channels,
                                     2 * kernel_size * kernel_size,
                                     kernel_size=kernel_size,
                                     stride=stride,
                                     padding=self.padding,
                                     bias=True)

        nn.init.constant_(self.offset_conv.weight, 0.)
        nn.init.constant_(self.offset_conv.bias, 0.)

        self.modulator_conv = nn.Conv2d(in_channels,
                                     1 * kernel_size * kernel_size,
                                     kernel_size=kernel_size,
                                     stride=stride,
                                     padding=self.padding,
                                     bias=True)

        nn.init.constant_(self.modulator_conv.weight, 0.)
        nn.init.constant_(self.modulator_conv.bias, 0.)

        self.regular_conv = nn.Conv2d(in_channels=in_channels,
                                      out_channels=out_channels,
                                      kernel_size=kernel_size,
                                      stride=stride,
                                      padding=self.padding,
                                      bias=bias)

    def forward(self, x):
        h, w = x.shape[2:]
        max_offset = max(h, w)/4.

        offset = self.offset_conv(x).clamp(-max_offset, max_offset)
        modulator = 2. * torch.sigmoid(self.modulator_conv(x))

        x = torchvision.ops.deform_conv2d(input=x,
                                          offset=offset,
                                          weight=self.regular_conv.weight,
                                          bias=self.regular_conv.bias,
                                          padding=self.padding,
                                          mask=modulator
                                          )
        return x


## Deformable CNN class

In [59]:


class DeformableCNN(nn.Module):
    def __init__(self, num_classes):
        super(DeformableCNN, self).__init__()
        self.conv1 = DeformableConv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = DeformableConv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = DeformableConv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv4 = DeformableConv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((14, 14))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.adaptive_pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = torch.sigmoid(x)
        return x


## train Deformable CNN

In [60]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def_cnn_model = DeformableCNN(num_classes=90)
def_cnn_model=def_cnn_model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(def_cnn_model.parameters(), lr=0.0002)

num_epochs = 30

start_time = time.time()
for epoch in range(num_epochs):
    running_loss = 0
    correct=0
    total=0


    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        inputs=inputs.to(device)
        labels=labels.to(device)

        outputs = def_cnn_model(inputs)
        predicted = torch.round(outputs)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    accuracy_train = 100 * correct / (total*90)

    print(f'Epoch [{epoch + 1}], Accuracy_training:{accuracy_train:.4f}%')
    print(f'Epoch [{epoch + 1}], Loss: {running_loss / total:.2}')

end_time = time.time()
execution_time = end_time - start_time
print(f"Total training time: {execution_time:.2f} seconds")


Epoch [1], Accuracy_training:75.3556%
Epoch [1], Loss: 0.79
Epoch [2], Accuracy_training:82.3789%
Epoch [2], Loss: 0.79
Epoch [3], Accuracy_training:83.7833%
Epoch [3], Loss: 0.78
Epoch [4], Accuracy_training:83.5711%
Epoch [4], Loss: 0.78
Epoch [5], Accuracy_training:83.5100%
Epoch [5], Loss: 0.78
Epoch [6], Accuracy_training:82.8633%
Epoch [6], Loss: 0.77
Epoch [7], Accuracy_training:82.1433%
Epoch [7], Loss: 0.77
Epoch [8], Accuracy_training:83.3133%
Epoch [8], Loss: 0.77
Epoch [9], Accuracy_training:84.4711%
Epoch [9], Loss: 0.76
Epoch [10], Accuracy_training:84.9578%
Epoch [10], Loss: 0.76
Epoch [11], Accuracy_training:83.5767%
Epoch [11], Loss: 0.76
Epoch [12], Accuracy_training:83.6522%
Epoch [12], Loss: 0.76
Epoch [13], Accuracy_training:85.0689%
Epoch [13], Loss: 0.75
Epoch [14], Accuracy_training:86.1011%
Epoch [14], Loss: 0.75
Epoch [15], Accuracy_training:87.1089%
Epoch [15], Loss: 0.75
Epoch [16], Accuracy_training:87.2000%
Epoch [16], Loss: 0.75
Epoch [17], Accuracy_train

In [61]:
correct = 0
total = 0
test_loss=0
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        inputs, labels = data
        inputs=inputs.to(device)
        labels=labels.to(device)
        outputs = def_cnn_model(inputs)
        predicted = torch.round(outputs)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        loss = criterion(outputs, labels)
        test_loss += loss.item()
    accuracy_test = 100 * correct / (total*90)

    print(f'Accuracy_test:{accuracy_test:.4f}%')
    print(f'Loss: {running_loss / total:.2}')


Accuracy_test:91.5222%
Loss: 2.4


<div dir=rtl>

مشاهده می شود که زمان انجام کانولوشن دفورمیبل بیشتر است و منطقی هم هست چرا که هر لایه کانولوشن دفورموبل خود شامل سه لایه کانولوشن معمولی و یکسری محاسبات دیگر و همچنین تعداد پارامترهای بیشتری نیز هست.

همچنین دقت به دست آمده برای یادگیری در کانولوشن معمولی پایین تر از دفورمبل است ولی تست بالاتر از آن.

لازم به ذکر است در تمام موارد دقت را برابر کل کلاس های شناسایی شده قرار دادیم.یعنی در نظر گرفتیم که از بین کلاس هایی که در بردار وان هات یک هستند کدام تشخیص داده شده است و در نهایت نیز تعداد کل را برابر تعداد ضرب در 90 در نظر گرفتیم.

این موضوع به دلیل تعریف دقت بر روی داده ها است احتمالا.چون هر تصویر ممکن است شامل چند کلاس باشد و ما فقط قید بودن یا نبودن آن را در نظر گرفتیم.در جایی که اگر تعداد آنها را هم لحاظ میکردیم دقت دفورمبل تفاوت بیشتری میکرد.